In [2]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.matutils import Sparse2Corpus

pyLDAvis.enable_notebook()
nltk.download('wordnet')
nltk.download('stopwords')
#sklearn.decomposition.LatentDirichletAllocation

[nltk_data] Downloading package wordnet to /home/bruno/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/bruno/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
data = pd.read_csv('data.csv')
data.fillna('', inplace=True)
data.head()

,contest,problem_name,problem_statement,problem_tags
0,325,A,You are given n rectangles. The corners of rec...,"implementation,*1500"
1,325,B,Daniel is organizing a football tournament. He...,"binarysearch,math,*1800"
2,325,C,Piegirl has found a monster and a book about m...,"dfsandsimilar,graphs,shortestpaths,*2600"
3,325,D,"In a far away land, there exists a planet shap...","dsu,*2900"
4,325,E,Piegirl found the red button. You have one las...,"combinatorics,dfsandsimilar,dsu,graphs,greedy,..."


In [4]:
print(f'quantidade de problemas: {data.shape[0]}')
print(f'quantidade de contests: {len(data.contest.unique())}')
print(f'problemas repetidos: {data.shape[0] - len(data.problem_statement.unique())}')

data.drop_duplicates(subset='problem_statement', inplace=True)

quantidade de problemas: 8343
quantidade de contests: 1424
problemas repetidos: 1524


## Pre-processamento

In [5]:
sw = stopwords.words('english')
sw.append(['input', 'output', 'th'])
lemma = WordNetLemmatizer()
text = data.problem_statement

# Problem_statement
text = text.apply(lambda x: re.sub('[,|.|(|)|$|!|?|!]',' ',x)) # removendo caracteres especiais
text = text.apply(lambda x: [i for i in x.split() if i.lower() not in sw]) # removendo stop-words
text = text.apply(lambda x: ' '.join([lemma.lemmatize(i) for i in x])) # lemmatizando todo

# Topics
topics = data.problem_tags
topics = topics.apply(lambda x: re.sub('[*][0-9]+','',x)) # remove os ratings *800 etc
topics = topics.apply(lambda x: [i for i in x.split(',') if i != '']) # tira strings vazias

In [6]:
text

0       given n rectangle corner rectangle integer coo...
1       Daniel organizing football tournament come fol...
2       Piegirl found monster book monster pie reading...
3       far away land exists planet shaped like cylind...
4       Piegirl found red button one last chance chang...
                              ...                        
8338    n block arranged row numbered left right start...
8339    map capital Berland viewed infinite coordinate...
8340    play strategic video game yeah ran good proble...
8341    first let's define function f x follows: \begi...
8342    Recently lot student enrolled Berland State Un...
Name: problem_statement, Length: 6819, dtype: object

In [7]:
topics

0                                        [implementation]
1                                    [binarysearch, math]
2                  [dfsandsimilar, graphs, shortestpaths]
3                                                   [dsu]
4       [combinatorics, dfsandsimilar, dsu, graphs, gr...
                              ...                        
8338                                       [greedy, math]
8339       [bruteforce, geometry, greedy, implementation]
8340    [datastructures, dp, greedy, implementation, s...
8341              [binarysearch, combinatorics, dp, math]
8342                                         [bruteforce]
Name: problem_tags, Length: 6819, dtype: object

In [34]:
lista = topics.to_list()

d = dict()
for x in lista:
    for y in x:
        if y not in d:
            d[y]= 1
        else:
            d[y]+=1

x= sorted(d.items(), key=lambda item:item[1], reverse=True)

for i in x:
    print(i)



('implementation', 1970)
('math', 1733)
('greedy', 1645)
('dp', 1373)
('datastructures', 1076)
('bruteforce', 1020)
('constructivealgorithms', 981)
('graphs', 720)
('sortings', 658)
('binarysearch', 652)
('dfsandsimilar', 590)
('trees', 511)
('strings', 484)
('numbertheory', 453)
('combinatorics', 390)
('*specialproblem', 367)
('twopointers', 310)
('bitmasks', 300)
('geometry', 291)
('dsu', 210)
('shortestpaths', 174)
('divideandconquer', 167)
('probabilities', 162)
('hashing', 136)
('games', 131)
('interactive', 114)
('flows', 99)
('matrices', 87)
('stringsuffixstructures', 64)
('fft', 56)
('graphmatchings', 54)
('ternarysearch', 40)
('meet-in-the-middle', 33)
('expressionparsing', 32)
('2-sat', 19)
('chineseremaindertheorem', 10)
('schedules', 5)


## Vetorizando

In [ ]:
max_df = 0.8
min_df = 0.05
vec = CountVectorizer(max_df=max_df, min_df=min_df, ngram_range=(1,3))

In [ ]:
def vect2gensim(vectorizer, dtmatrix):
    # transform sparse matrix into gensim corpus and dictionary
    corpus_vect_gensim = Sparse2Corpus(dtmatrix, documents_columns=False)
    dictionary = Dictionary.from_corpus(corpus_vect_gensim,
            id2word=dict((id, word) for word, id in vectorizer.vocabulary_.items()))
    return (corpus_vect_gensim, dictionary)



In [ ]:
X = vec.fit_transform(text)
X.shape
X = vect2gensim(vec, X)

## Agrupando

In [ ]:
# Pegando quantidade de topicos

total_topics = set()
topics = topics.apply(lambda x: [total_topics.add(i) for i in x])

In [ ]:
n_total_topics = len(total_topics)
print(f'Número de Tópicos: {n_total_topics}')
total_topics

Número de Tópicos: 37


{'*specialproblem',
 '2-sat',
 'binarysearch',
 'bitmasks',
 'bruteforce',
 'chineseremaindertheorem',
 'combinatorics',
 'constructivealgorithms',
 'datastructures',
 'dfsandsimilar',
 'divideandconquer',
 'dp',
 'dsu',
 'expressionparsing',
 'fft',
 'flows',
 'games',
 'geometry',
 'graphmatchings',
 'graphs',
 'greedy',
 'hashing',
 'implementation',
 'interactive',
 'math',
 'matrices',
 'meet-in-the-middle',
 'numbertheory',
 'probabilities',
 'schedules',
 'shortestpaths',
 'sortings',
 'strings',
 'stringsuffixstructures',
 'ternarysearch',
 'trees',
 'twopointers'}

In [ ]:
n_components = n_total_topics
max_iter = 100

id2word = X[1]
corpus = X[0]

lda = LdaModel(corpus=corpus, num_topics=n_components, id2word=id2word, iterations=max_iter)

In [ ]:
# lda_matrix = lda.fit_transform(X)



In [ ]:
# top_word = lda.components_
# terms = vec.get_feature_names_out()

# for i, comp in enumerate(top_word):
#     top_terms_key = zip(terms, comp)
#     top_terms_key= sorted(top_terms_key, key = lambda t: t[1], reverse=True)[:10]
#     top_terms_list= list(dict(top_terms_key).keys())
#     print("Topic "+str(i)+": ",top_terms_list)

## Validando

In [ ]:
new_model = CoherenceModel(model=lda, texts=text, corpus=corpus, dictionary=id2word, coherence="u_mass")
print(new_model.get_coherence())

# todo -> entender metrica de coerencia
#      -> usar dentro dos atributos o que ja foi calculado no CountVectorizer

-1.1123019473354472


## Visualizando

In [ ]:
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

panel = pyLDAvis.gensim.prepare(lda, corpus, dictionary=id2word)
panel
# todo 
# mudar para o modelo do gensim

/home/bruno/Desktop/PIBIC/venv/lib/python3.10/site-packages/pyLDAvis/_prepare.py:244: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
29     0.078682 -0.021163       1        1  5.880614
18    -0.155631  0.096572       2        1  5.725926
23     0.017267 -0.080771       3        1  5.618714
19    -0.240108 -0.013350       4        1  5.208090
16     0.069621 -0.013782       5        1  4.215332
31    -0.025613  0.049926       6        1  3.829755
25     0.094903 -0.027775       7        1  3.720773
30    -0.116182  0.040408       8        1  3.684097
7     -0.217454  0.030378       9        1  3.635557
9     -0.178015 -0.021855      10        1  3.393070
21     0.087702 -0.067473      11        1  3.322009
24     0.071291  0.022450      12        1  3.211858
12     0.055648 -0.051729      13        1  3.191295
4      0.082302 -0.027089      14        1  3.075921
32     0.079899 -0.021246      15        1  3.002546
26     0.041107 -0.050581      16        1  2.710690
35     0.075973  0.150239      17        1  2.674641
0      0.086259 -0.017236      18        1  2.608364
14    -0.004844  0.001221      19        1  2.607645
10     0.039592  0.017012      20        1  2.420856
33     0.089866 -0.008089      21        1  2.245091
1      0.036724 -0.076271      22        1  2.182996
17    -0.121044 -0.065889      23        1  2.151173
11     0.053238  0.073200      24        1  1.966577
5     -0.019299  0.175181      25        1  1.939109
36    -0.128232 -0.020015      26        1  1.852144
34     0.081447  0.001617      27        1  1.849093
13     0.055183  0.028971      28        1  1.687418
20     0.049306 -0.069534      29        1  1.681920
15    -0.066879  0.007053      30        1  1.607475
27     0.003749 -0.044438      31        1  1.440202
3     -0.083136 -0.059902      32        1  1.384759
28     0.027346  0.012635      33        1  1.308342
6     -0.070716 -0.102129      34        1  0.872262
22     0.023854  0.106560      35        1  0.861402
2      0.122623  0.078317      36        1  0.762095
8      0.003572 -0.031424      37        1  0.470187, topic_info=          Term          Freq         Total Category  logprob  loglift
235     string   6855.000000   6855.000000  Default  30.0000  30.0000
329      query   3984.000000   3984.000000  Default  29.0000  29.0000
285      array   4301.000000   4301.000000  Default  28.0000  28.0000
224         le  13888.000000  13888.000000  Default  27.0000  27.0000
4        point   3409.000000   3409.000000  Default  26.0000  26.0000
..         ...           ...           ...      ...      ...      ...
53    possible     22.491774   4983.883171  Topic37  -4.8627  -0.0410
315  test case     22.258043   4862.500936  Topic37  -4.8731  -0.0268
167     answer     21.761309   4843.591948  Topic37  -4.8957  -0.0455
212     length     19.637489   4189.978145  Topic37  -4.9984  -0.0032
15        next     17.992013   3682.101701  Topic37  -5.0859   0.0385

[1840 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
382       1  0.286353   000
382       2  0.003923   000
382       3  0.074530   000
382       4  0.013729   000
382       5  0.120948   000
...     ...       ...   ...
65       32  0.032420  zero
65       33  0.006343  zero
65       34  0.078232  zero
65       36  0.001410  zero
65       37  0.016915  zero

[12928 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[30, 19, 24, 20, 17, 32, 26, 31, 8, 10, 22, 25, 13, 5, 33, 27, 36, 1, 15, 11, 34, 2, 18, 12, 6, 37, 35, 14, 21, 16, 28, 4, 29, 7, 23, 3, 9])